# **텍스트데이터분석 프로젝트 크롤링 코드**
#### **주제** : 게임사 5곳 비교분석 (넥슨, 크래프톤, 넷마블, 라이엇코리아, 엔씨소프트)

#### **1. API를 활용한 크롤링 (링크, 제목, 간단한 설명)**
- 기본 틀 설계 이후 키워드를 수정해가면서 반복적으로 데이터를 수집하였습니다.
- 네이버 뉴스 크롤링

In [1]:
import urllib.request
import urllib.parse

## API 인증 정보
client_id = "Pw1gBeWqBVD7KGk7_VP_"
client_secret = "rSCeFH8jY1"

## 검색 기준 설정
n_display = 100  ## API 최대 요청 가능 수
news_list = []  ## 결과를 저장할 리스트
base_url = 'https://openapi.naver.com/v1/search/news.json'  ## 네이버 뉴스 base url

queries_krafton = ['크래프톤', '크래프톤 게임', '크래프톤 주가', '크래프톤 채용', '크래프톤 복지']
queries_nexon = ['넥슨', '넥슨 게임즈', '넥슨 주가', '넥슨 채용', '넥슨 복지']
queries_netmable = ['넷마블', '넷마블 게임', '넷마블 주가', '넷마블 채용', '넷마블 복지']
queries_ncsoft = ['엔씨소프트', '엔씨소프트 게임', '엔씨소프트 주가', '엔씨소프트 채용', '엔씨소프트 복지']
queries_riot = ['라이엇게임즈', '라이엇코리아', '라이엇게임즈 주가', '라이엇게임즈 채용', '라이엇게임즈 복지']

## 각 검색어에 대해 처리
for query in queries_ncsoft:
    start_index = 1
    while start_index <= 1000:  ## 최대 1000개 기사 수집
        encQuery = urllib.parse.quote(query)
        url = f"{base_url}?query={encQuery}&display={n_display}&start={start_index}&sort=sim"
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)

        with urllib.request.urlopen(request) as response:
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read()
                search_results = eval(response_body.decode('utf-8'))
                items = search_results['items']
                
                ## 네이버 뉴스만 추출 --> 'naver.com'를 포함하는 경우만
                for item in items:
                    if 'naver.com' in item['link']:
                        news_list.append(item)
            else:
                print(f"Error Code: {rescode}")
        start_index += n_display  ## 다음 페이지 인덱스 증가
        
## 수집된 기사 수 출력
print(f"Collected {len(news_list)} articles that include 'naver.com' in the link.")

Collected 1599 articles that include 'naver.com' in the link.


- 네이버 블로그 크롤링

In [2]:
import urllib.request
import urllib.parse

## API 인증 정보
client_id = "Pw1gBeWqBVD7KGk7_VP_"
client_secret = "rSCeFH8jY1"

n_display = 100  ## API 최대 요청 가능 수
blog_list = []  ## 결과를 저장할 리스트
base_url = 'https://openapi.naver.com/v1/search/blog.json'  ## 네이버 블로그 base url

## 각 검색어에 대해 처리
for query in queries_ncsoft:
    start_index = 1
    while start_index <= 1000:  ## 최대 1000개 글 수집
        encQuery = urllib.parse.quote(query)
        url = f"{base_url}?query={encQuery}&display={n_display}&start={start_index}&sort=sim"
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)

        with urllib.request.urlopen(request) as response:
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read()
                search_results = eval(response_body.decode('utf-8'))
                items = search_results['items']
                
                ## 네이버 뉴스만 추출 --> 'naver.com'를 포함하는 경우만
                for item in items:
                    if 'naver.com' in item['link']:
                        blog_list.append(item)
            else:
                print(f"Error Code: {rescode}")
        start_index += n_display  ## 다음 페이지 인덱스 증가

## 수집된 기사 수 출력
print(f"Collected {len(blog_list)} articles that include 'naver.com' in the link.")

Collected 4729 articles that include 'naver.com' in the link.


---
#### **2. 데이터프레임화 및 데이터 전처리**

In [3]:
import pandas as pd
import re
pd.set_option('display.max_colwidth', 1000)  ## 최대 1000개의 텍스트까지 보는 코드
news = pd.DataFrame(news_list)
blog = pd.DataFrame(blog_list)
news.shape, blog.shape

((1599, 5), (4729, 6))

In [4]:
## 중복 링크 제거 및 링크 column 클렌징
news = news.drop_duplicates(subset='link', keep='first')
news.link = news.link.apply(lambda x : str(x).replace('\\', ''))
blog = blog.drop_duplicates(subset='link', keep='first')
blog.link = blog.link.apply(lambda x : str(x).replace('\\', ''))

## 뉴스와 블로그 중 너무 오래된 글은 제거하고자 2015년 이상의 글만 수집
news.pubDate = news['pubDate'].apply(lambda x: pd.to_datetime(x))
news['year'] = news.pubDate.dt.year
blog.postdate = blog['postdate'].apply(lambda x: pd.to_datetime(x))
blog['year'] = blog.postdate.dt.year

news = news.loc[news.year >= 2015]
blog = blog.loc[blog.year >= 2015]

---
#### **3. selenium을 통한 body 크롤링**
- api를 활용하면 본문 내용이 제한되는 문제가 있어 selenium을 활용해 본문 내용을 추가 수집하였습니다
- news

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time
import random

driver = webdriver.Chrome()
driver.implicitly_wait(3)
news_contents = []
total_links = len(news.link)  ## 전체 링크 개수

for index, i in enumerate(news.link): 
    driver.get(i)
    time.sleep(random.uniform(1, 2))  ## 1~2초 사이의 균등분포로 time sleep 설정
    progress = (index + 1) / total_links * 100  ## 진행 정도를 알아보고자 progress 변수 생성

    ## 페이지 장르(경제 / 스포츠 등)에 따라서 태그가 달라지는 경우가 있어 다르게 크롤링 진행
    try:
        if 'n.news.naver.com' in i:
            tmp = driver.find_elements(By.ID, 'newsct_article')  ## 태그 탐색
            news_contents.append(tmp[0].text) if tmp else news_contents.append('사라진 페이지') 
        else:
            tmp = driver.find_elements(By.ID, 'comp_news_article')  ## 태그 탐색
            news_contents.append(tmp[0].text) if tmp else news_contents.append('사라진 페이지')
    except IndexError:
        news_contents.append('사라진 페이지')
    
    print(f"Progress: {progress:.2f}% Complete")  ## 진행 정도 출력

driver.quit()
print("본문 크롤링이 완료되었습니다.")
news['body'] = news_contents  ## news 데이터프레임의 body column에 내용을 할당

Progress: 0.08% Complete
Progress: 0.16% Complete
Progress: 0.24% Complete
Progress: 0.32% Complete
Progress: 0.40% Complete
Progress: 0.48% Complete
Progress: 0.56% Complete
Progress: 0.65% Complete
Progress: 0.73% Complete
Progress: 0.81% Complete
Progress: 0.89% Complete
Progress: 0.97% Complete
Progress: 1.05% Complete
Progress: 1.13% Complete
Progress: 1.21% Complete
Progress: 1.29% Complete
Progress: 1.37% Complete
Progress: 1.45% Complete
Progress: 1.53% Complete
Progress: 1.61% Complete
Progress: 1.69% Complete
Progress: 1.78% Complete
Progress: 1.86% Complete
Progress: 1.94% Complete
Progress: 2.02% Complete
Progress: 2.10% Complete
Progress: 2.18% Complete
Progress: 2.26% Complete
Progress: 2.34% Complete
Progress: 2.42% Complete
Progress: 2.50% Complete
Progress: 2.58% Complete
Progress: 2.66% Complete
Progress: 2.74% Complete
Progress: 2.82% Complete
Progress: 2.91% Complete
Progress: 2.99% Complete
Progress: 3.07% Complete
Progress: 3.15% Complete
Progress: 3.23% Complete


- blog는 news와 다르게 시간에 따라 태그가 달라지는 경우가 있어 추가 전처리를 해주었습니다
- blog

In [92]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import random

driver = webdriver.Chrome()
driver.implicitly_wait(3)
blog_contents = []
total_links = len(blog.link)  ## 전체 링크 개수

for index, i in enumerate(blog.link):
    driver.get(i)
    time.sleep(random.uniform(1, 2))  ## 1~2초 사이의 균등분포로 time sleep 설정
    progress = (index + 1) / total_links * 100  ## 진행 정도를 알아보고자 progress 변수 생성

    ## 네이버 블로그 작성 시점에 따른 버전 차이로 태그명이 달라져 경우에 따라 구분하여 크롤링 진행
    try:
        driver.switch_to.frame("mainFrame")  ## 메인 프레임에 스위치 접근
        a = driver.find_element(By.CSS_SELECTOR, 'div.se-main-container').text  ## 태그 탐색
        blog_contents.append(a)
    except NoSuchElementException:  ## 메인 프레임에서 태그가 없다면
        try:
            a = driver.find_element(By.CSS_SELECTOR, 'div#content-area').text  ## 예전 태그명으로 탐색
            blog_contents.append(a)
        except NoSuchElementException:  ## 태그를 찾지 못하면 사라진 페이지로 할당
            blog_contents.append('사라진 페이지')
    except Exception as e:  ## 위의 특정 예외 이외에 다른 모든 예외를 잡아 일괄 처리(추후 삭제 위해 임의로 사라진 페이지와 동일 취급)
        print(f"Error encountered: {e}")  ## 오류 메세지 출력
        blog_contents.append('사라진 페이지')

    print(f"Progress: {progress:.2f}% Complete")  ## 진행 정도 출력

driver.quit()
blog['body'] = blog_contents  ## news 데이터프레임의 body column에 내용을 할당

Progress: 0.03% Complete
Progress: 0.06% Complete
Progress: 0.09% Complete
Progress: 0.11% Complete
Progress: 0.14% Complete
Progress: 0.17% Complete
Progress: 0.20% Complete
Progress: 0.23% Complete
Progress: 0.26% Complete
Progress: 0.29% Complete
Progress: 0.31% Complete
Progress: 0.34% Complete
Progress: 0.37% Complete
Progress: 0.40% Complete
Progress: 0.43% Complete
Progress: 0.46% Complete
Progress: 0.49% Complete
Progress: 0.51% Complete
Progress: 0.54% Complete
Progress: 0.57% Complete
Progress: 0.60% Complete
Progress: 0.63% Complete
Progress: 0.66% Complete
Progress: 0.69% Complete
Progress: 0.72% Complete
Progress: 0.74% Complete
Progress: 0.77% Complete
Progress: 0.80% Complete
Progress: 0.83% Complete
Progress: 0.86% Complete
Progress: 0.89% Complete
Progress: 0.92% Complete
Progress: 0.94% Complete
Progress: 0.97% Complete
Progress: 1.00% Complete
Progress: 1.03% Complete
Progress: 1.06% Complete
Progress: 1.09% Complete
Progress: 1.12% Complete
Progress: 1.14% Complete


In [ ]:
## 데이터프레임 추출
news.to_csv('ncsoft_news.csv', index = False)
blog.to_csv('ncsoft_blog.csv', index = False)

In [6]:
## 위 틀을 기반으로 이름만 다르게 해서 데이터 수집하였습니다

## news.to_csv('krafton_news.csv', index = False)
## blog.to_csv('krafton_blog.csv', index = False)
## news.to_csv('nexon_news.csv', index = False)
## blog.to_csv('nexon_blog.csv', index = False)
## news.to_csv('netmarble_news.csv', index = False)
## blog.to_csv('netmarble_blog.csv', index = False)
## news.to_csv('ncsoft_news.csv', index = False)
## blog.to_csv('ncsoft_blog.csv', index = False)
## news.to_csv('riot_news.csv', index = False)
## blog.to_csv('riot_blog.csv', index = False)